# Explorando bases vetoriais

In [1]:
from langchain.document_loaders import PyPDFLoader

pdf_path = 'data/PPC_TSI_EaD.pdf'
loader = PyPDFLoader(pdf_path)
pages = loader.load_and_split()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    #chunk_size=200,
    chunk_size=1000,
    chunk_overlap=50,
    length_function=len,
)

texts = text_splitter.split_documents(pages)

print(texts[0])

page_content='Ministério da Educação
Instituto Federal de Educação, Ciência e Tecnologia do Rio Grande do Norte
DELIBERAÇÃO Nº 59/2022 - CONSEPEX/IFRN DELIBERAÇÃO Nº 59/2022 - CONSEPEX/IFRN' metadata={'source': 'data/PPC_TSI_EaD.pdf', 'page': 0}


In [3]:
print (f"Quantidade de chunks: {len(texts)} ")

Quantidade de chunks: 3183 


In [4]:
#!pip install faiss-cpu
#pip install faiss-gpu

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.from_documents(texts,  OllamaEmbeddings(model="mxbai-embed-large"))

/var/folders/fh/g5fgxqq513n014czxdxy7sfc0000gn/T/ipykernel_82879/4139363345.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  db = FAISS.from_documents(texts,  OllamaEmbeddings(model="mxbai-embed-large"))


disciplinas do curso.


In [16]:
query = "Qual o nome do curso?"
docs = db.similarity_search(query)
print(docs[0].page_content)

4 
SUMÁRIO 
 
 
1. IDENTIFICAÇÃO DO CURSO 5  
2. DADOS DO COORDENADOR DO CURSO 5  
3. DESCRIÇÃO DA OFERTA 5  
4. JUSTIFICATIVA DA OFERTA PARA DESENVOLVIMENTO LOCAL 5  
5. APOIO AO DISCENTE 7


In [17]:
# Perform similarity search to get the top 5 most relevant chunks (by similarity)
docs = db.similarity_search(query, k=5)

# Print the content of each of the top 5 chunks
for i, doc in enumerate(docs):
    print(f"Chunk {i + 1}: {doc.page_content}")

Chunk 1: 4 
SUMÁRIO 
 
 
1. IDENTIFICAÇÃO DO CURSO 5  
2. DADOS DO COORDENADOR DO CURSO 5  
3. DESCRIÇÃO DA OFERTA 5  
4. JUSTIFICATIVA DA OFERTA PARA DESENVOLVIMENTO LOCAL 5  
5. APOIO AO DISCENTE 7
Chunk 2: EMENTA 
Consultas avançadas em linguagem de consulta estruturada (SQL); elementos que proporcionam me lhor
Chunk 3: A denominação do curso está adequada ao Catálogo Nacional dos 
Cursos Superiores de Tecnologia? 
NSA para demais cursos 
SIM
Chunk 4: disponíveis na biblioteca para funcionamento do curso por disciplina.
Chunk 5: g) Acervo para o funcionamento do curso: Os dados relativos a descrição e quan titativo de 
títulos da bibliografia básica, complementar que estão disponíveis na biblioteca par a


# Explorando retriever

In [18]:
from langchain_ollama.llms import OllamaLLM
from langchain.chains import RetrievalQA

# Create the LLM model and retriever
model = OllamaLLM(model="llama3.2:latest")
retriever = db.as_retriever()

# Create a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(llm=model, retriever=retriever, chain_type="stuff")

In [19]:
# Example usage of the RetrievalQA chain
response = qa_chain.invoke(query)
print("QA Response:", response)

QA Response: {'query': 'Qual o nome do curso?', 'result': 'Não sei qual é o nome do curso.'}
